## How to plot large graphs

In [ ]:
from random import choice

import networkx as nx
import numpy as np
from tqdm.auto import tqdm

In [ ]:
categories = "abcdefghijk"
node_categories = "12345"

G = nx.erdos_renyi_graph(n=3000, p=0.1)
for u, v in tqdm(G.edges()):
    G.edges[u, v]["group"] = choice(categories)
    G.edges[u, v]["edge_val"] = np.random.exponential()

for n in tqdm(G.nodes()):
    G.nodes[n]["category"] = choice(node_categories)
    G.nodes[n]["value"] = np.random.normal()

That graph above takes a lot of time to plot.
I'm not even going to try :).

In [ ]:
import nxviz as nv

The key difficulty here is that we have lots of individual glyphs to plot to the screen.
Instead of doing that, we could take advantage of higher order metadata.
One particular way that we can draw the network
is by mapping out the strength of connections between
_groups of nodes_ rather than between individual nodes.

In [ ]:
from nxviz import utils

In [ ]:
nt = utils.node_table(G)

In [ ]:
et = utils.edge_table(G)

To draw large graphs using the same design ideas as the circos plots,
but made scalable instead, we will take advantage of known node groupings
to produce an informative plot.
The first piece of information that we need
is the proportion of each node category.

In [ ]:
import pandas as pd

proportions = nt["category"].value_counts(normalize=True).sort_index()
end_angles = proportions.cumsum() * 360
start_angles = end_angles - proportions * 360
start_angles, end_angles

We also need to prepare the colors.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches

from nxviz import aesthetics as aes
from nxviz.plots import aspect_equal, despine, rescale

In [ ]:
colors = aes.data_color(pd.Series(start_angles.index))
colors.index = start_angles.index


plot_data = pd.DataFrame(
    {"start_angle": start_angles, "end_angle": end_angles, "color": colors}
)
plot_data

In [ ]:
fig, ax = plt.subplots()

for d in plot_data.itertuples(index=False):
    w = patches.Wedge(
        center=(0, 0),
        r=10,
        theta1=d.start_angle,
        theta2=d.end_angle,
        color=d.color,
        width=0.5,
    )
    ax.add_patch(w)


# Draw one path patch


ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

aspect_equal()
despine()

Now, we have to draw in edges as _bands with area_ rather than as lines.
With these bands, we start by identifying the edges
that originate from one group of nodes and conclude in other groups of nodes.
Using this information, we can then count the number of edges, and thus the proportion of edges.

In [ ]:
get_cartesian(0, theta=(target_end + 2 * np.pi) / 2)

In [ ]:
def clockwise_angle(theta1, theta2):
    """Obtain the clockwise angle to go from theta1 to theta2."""
    if theta2 < theta1:
        theta2 = 2 * np.pi + theta2
    return theta2 - theta1

In [ ]:
target_end + clockwise_angle(target_end, source_start) / 2

In [ ]:
angle = target_end + clockwise_angle(target_end, source_start) / 2

In [ ]:
get_cartesian(0, theta=angle)

In [ ]:
get_cartesian(1, 2 * np.pi)

In [ ]:
get_cartesian(
    0.1, theta=target_end + clockwise_angle(target_end, source_start) / 2
)

In [ ]:
from matplotlib.path import Path

In [ ]:
r = 0.8
source_start = 0
source_end = np.pi / 4
target_start = 3 * np.pi / 4
target_end = 3 * np.pi / 2

p1 = Path.arc(theta1=np.rad2deg(source_start), theta2=np.rad2deg(source_end))
p2 = Path.arc(theta1=np.rad2deg(target_start), theta2=np.rad2deg(target_end))
p1._vertices, p1._codes, p2._vertices, p2._codes

The arc vertices can be multiplied by the radius `r` to get the actual arc radius.

In [ ]:


fig, ax = plt.subplots()
ax.add_patch(fill)
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)

aspect_equal()
despine()

In [ ]:
from matplotlib.path import Path

from nxviz.geometry import get_cartesian


def circos_bundle_patch(
    r: float,
    source_start: float,
    source_end: float,
    target_start: float,
    target_end: float,
) -> patches.PathPatch:
    """Return the matplotlib patch that draws a circos edge bundle.
    
    A circos edge bundle is defined as a bundle of edges
    that are drawn from one node to another. 
    
    ## Parameters:
    
    - `r`: Circos plot radius.
    - `source_start`: Start angle for source group, in radians.
    - `source_end`: End angle for source group, in radians.
    - `target_start`: Start angle for target group, in radians.
    - `target_end`: End angle for target group, in radians.
    """
    
    vertices = []
    codes = []
    p1 = Path.arc(theta1=np.rad2deg(source_start), theta2=np.rad2deg(source_end))
    vertices.extend(p1._vertices * r)
    codes.extend(p1._codes)

    source_target_midpoint = get_cartesian(
        0.0,
        theta=source_end + clockwise_angle(source_end, target_start) / 2,
    )
    vertices.append(source_target_midpoint)
    codes.append(Path.CURVE3)

    target_start_point = get_cartesian(r, theta=target_start)
    vertices.append(target_start_point)
    codes.append(Path.CURVE3)

    p2 = Path.arc(theta1=np.rad2deg(target_start), theta2=np.rad2deg(target_end))
    vertices.extend(p2._vertices * r)
    codes.extend(p2._codes)

    target_source_midpoint = get_cartesian(
        0.0,
        theta=target_end + clockwise_angle(target_end, source_start) / 2,
    )
    vertices.append(target_source_midpoint)
    codes.append(Path.CURVE3)

    source_start_point = get_cartesian(r, theta=source_start)
    vertices.append(source_start_point)
    codes.append(Path.CURVE3)

    path = Path(vertices, codes)
    fill = patches.PathPatch(path, alpha=0.1, ec="none")
    return fill

In [ ]:
fig, ax = plt.subplots()


#### These define the inputs
r = 0.8
source_start = 0
source_end = np.pi / 4
target_start = 3 * np.pi / 4
target_end = 3 * np.pi / 2

#### Next question is: how do we calculate source_start, source_end, target_start, and target_end?

fill = circos_bundle_patch(r, source_start, source_end, target_start, target_end)
ax.add_patch(fill)
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])

aspect_equal()
despine()

Now, let me try combining the two together.

In [ ]:
fig, ax = plt.subplots()


#### These define the inputs
r = 10
source_start = 0
source_end = np.pi / 4
target_start = 3 * np.pi / 4
target_end = 3 * np.pi / 2

#### Next question is: how do we calculate source_start, source_end, target_start, and target_end?

for d in plot_data.itertuples(index=False):
    w = patches.Wedge(
        center=(0, 0),
        r=r,
        theta1=d.start_angle,
        theta2=d.end_angle,
        color=d.color,
        width=0.5,
    )
    ax.add_patch(w)

fill = circos_bundle_patch(r-0.5, source_start, source_end, target_start, target_end)
ax.add_patch(fill)
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])

aspect_equal()

ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

aspect_equal()
# despine()

OK, let me summarize what that function above has done.

We have leveraged the matplotlib Path object to create paths.
Each path is parameterized by a set of vertices (i.e. points) and movement codes.
You can think of it as being like a pen that is commanded to move to a certain point,
then asked to draw lines to another point.
Arcs, which occur along the circumference of the circle, are not trivial to draw using Bezier curves,
so we use `Path.arc` to draw those, scaling the vertices by the radius desired.
Bezier curves, by contrast, are used to draw the internal lines.

OK, now we have to solve the problem of drawing each bundle of edges.
The proportion of edges between each group should dictate the arcs, I think.

In [ ]:
nt_idx = nt.reset_index().rename({"index": "node"}, axis=1)
grouped_edge_counts = (
    et.merge(nt_idx, left_on="source", right_on="node")
    .rename({"category": "source_category"}, axis=1)
    .merge(nt_idx, left_on="target", right_on="node")
    .rename({"category": "target_category"}, axis=1)
    .groupby(["source_category", "target_category"])
    .count()
)["group"]


# division by two (i.e. multiply by 180) necessary because each edge occupies a fraction of the whole circle twice
# - once for the source
# - once for the target
edge_arc_angles = (grouped_edge_counts / grouped_edge_counts.sum()) * 180
edge_arc_angles